# Capstone Project - **Concierge Agent** Category 

## *CoMailAgent* – Automated Resume Tailoring & Cold Email Job Outreach Agent 
- by Supriya & Sanya

Find below the code & instructions for this project

### Step 1: Configuring Google API Keys
This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

### Step 2: Import ADK components

Now, importing the specific components we will need from the Agent Development Kit and the Generative AI library. This keeps the code organized and ensures we have access to the necessary building blocks.

### Step 3: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [9]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

1. root_agent - Uses the 
1. get_task_items() tool
    * Goes through the db and returns a dictionary with 
2. cover_letter_agent - for writing a custom cover letter
    * **Extracts keywords and role expectations** from the job description.
    * ~~**Tailors the resume** to highlight relevant skills and achievements.~~
    * **Generates a professional, customized cover letter** for each job.
3. email_agent - for drafting a recruiter outreach email
    * **Creates a personalized cold outreach email** addressed to the recruiter or hiring manager.
    * **Packages attachments** (tailored resume + cover letter).
    * **Sends the email automatically**, or optionally requests user approval before sending.

In [46]:
# ----------------------------------------------
# CoMail AI Multi-Agent Pipeline (No Resume)
# ----------------------------------------------
import os
import io
from typing import Dict, Any

import pandas as pd
import requests
from fpdf import FPDF



In [48]:
# ------------------ Secrets ------------------
# Kaggle secrets helper (optional)
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
except Exception:
    user_secrets = None

def get_secret(name: str, default=None):
    if user_secrets:
        try:
            return user_secrets.get_secret(name)
        except Exception:
            pass
    return os.environ.get(name, default)

SPREADSHEET_ID = get_secret("SPREADSHEET_ID")
FILE_ID = get_secret("FILE_ID")
print(f"Secrets present: SPREADSHEET_ID={'set' if SPREADSHEET_ID else 'missing'}, FILE_ID={'set' if FILE_ID else 'missing'}")


Secrets present: SPREADSHEET_ID=set, FILE_ID=set


In [49]:
# ------------------ Utilities ----------------
def clean_text(text: str) -> str:
    """Remove BOM and replace non-ASCII chars with '?', safe for FPDF Latin-1"""
    if not text:
        return ""
    text = text.replace("\ufeff", "")
    return text.encode("ascii", "replace").decode()

def generate_pdf_bytes_from_text(text_content: str) -> bytes:
    """Generate PDF bytes from text (FPDF)"""
    text_content = clean_text(text_content)
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in text_content.splitlines():
        if line.strip() == "":
            pdf.ln(6)
        else:
            pdf.multi_cell(0, 8, txt=line)
    out = pdf.output(dest="S")
    if isinstance(out, str):
        return out.encode("latin1")
    return out



In [50]:
# ------------------ Tools -------------------
def get_job_data_method() -> Dict[str, Any]:
    url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=xlsx"
    try:
        df = pd.read_excel(url, sheet_name=0)
        if df.empty:
            return {"status": "error", "error_message": "Google Sheet is empty"}
        df.columns = [c.strip() for c in df.columns]
        job_data = df.to_dict(orient="list")
        return {"status": "success", "job_data": job_data}
    except Exception as e:
        return {"status": "error", "error_message": f"Failed to fetch sheet: {e}"}

def get_cover_letter_method() -> Dict[str, Any]:
    url = f"https://docs.google.com/document/d/{FILE_ID}/export?format=txt"
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        text = resp.text
        if not text.strip():
            return {"status": "error", "error_message": "Cover letter doc is empty"}
        return {"status": "success", "cover_letter": text}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error_message": f"HTTP error fetching doc: {e}"}

def update_cover_letter_method(cover_letter_text: str, job_description: str) -> Dict[str, Any]:
    """Simple deterministic update based on top keywords in job description"""
    if not cover_letter_text:
        return {"status": "error", "error_message": "No cover_letter_text provided"}
    if not job_description:
        return {"status": "error", "error_message": "No job_description provided"}
    # Extract keywords
    stopwords = {"the","and","with","that","from","this","will","your","for","you","are","have","has",
                 "a","an","to","in","on","of","by","as","be","or","is","it","at"}
    words = [w.strip(".,()[]:;\"'").lower() for w in job_description.split()]
    candidates = [w for w in words if len(w) > 4 and w not in stopwords]
    freq = {}
    for w in candidates:
        freq[w] = freq.get(w,0)+1
    top = [k for k,_ in sorted(freq.items(), key=lambda kv: kv[1], reverse=True)][:5]
    tailored_phrase = ""
    if top:
        tailored_phrase = f"I am particularly excited about this role's focus on {', '.join(top[:3])}.\n\n"
    # Insert tailored_phrase after first paragraph
    paragraphs = cover_letter_text.split("\n\n")
    if paragraphs:
        paragraphs[0] = paragraphs[0].strip() + "\n\n" + tailored_phrase.strip()
    updated = "\n\n".join(paragraphs)
    return {"status": "success", "updated_cover_letter": updated}

def save_text_as_pdf_method(content_to_save: str, filename: str = "cover_letter.pdf") -> Dict[str, Any]:
    if not content_to_save:
        return {"status": "error", "error_message": "No content to save"}
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"
    try:
        pdf_bytes = generate_pdf_bytes_from_text(content_to_save)
        path = os.path.join(os.getcwd(), filename)
        with open(path, "wb") as f:
            f.write(pdf_bytes)
        return {"status": "success", "file_path": path}
    except Exception as e:
        return {"status": "error", "error_message": f"PDF save failed: {e}"}

def prepare_and_print_email_method(recruiter_email: str, recruiter_name: str, job_title: str, email_body: str, pdf_path: str) -> Dict[str, Any]:
    greeting = f"Dear {recruiter_name}," if recruiter_name else "Hello,"
    subject = f"Application for {job_title}"
    full_body = f"{greeting}\n\n{email_body}\n\nBest regards,\nSanya"
    print("\n" + "="*60)
    print("📧  EMAIL PREVIEW")
    print("="*60)
    print(f"To: {recruiter_email}")
    print(f"Subject: {subject}")
    print("\n--- Email Body ---\n")
    print(full_body)
    print("\n--- Attachment (cover letter PDF saved) ---")
    print(f"{pdf_path}")
    print("="*60 + "\n")
    return {"status": "success", "message": "Printed email preview", "pdf_path": pdf_path}



In [51]:
# ------------------ Orchestrator -------------------
def orchestrator_pipeline():
    out_all = []
    res_jobs = get_job_data_method()
    if res_jobs.get("status") != "success":
        print("ERROR fetching job data:", res_jobs.get("error_message"))
        return {"status": "error", "detail": res_jobs}
    job_data = res_jobs["job_data"]
    num_rows = len(next(iter(job_data.values())))
    print(f"Found {num_rows} job rows. Processing sequentially...")

    for i in range(num_rows):
        job_title = job_data.get("job_title", [""]*num_rows)[i]
        job_description = job_data.get("job_description", [""]*num_rows)[i]
        recruiter_name = job_data.get("recruiter_name", [""]*num_rows)[i]
        recruiter_email = job_data.get("recruiter_email", [""]*num_rows)[i]
        job_id = job_data.get("job_id", [str(i+1)])[i]

        print(f"\n--- Processing job {i+1}/{num_rows}: {job_title} (id={job_id}) ---")

        # Get base cover letter
        res_cl = get_cover_letter_method()
        if res_cl.get("status") != "success":
            print("ERROR fetching cover letter:", res_cl.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "get_cover_letter"})
            continue
        base_cover = res_cl["cover_letter"]

        # Update cover letter
        res_updated = update_cover_letter_method(base_cover, job_description)
        if res_updated.get("status") != "success":
            print("ERROR updating cover letter:", res_updated.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "update_cover_letter"})
            continue
        updated_cover = res_updated["updated_cover_letter"]

        # Save PDF (one per job)
        pdf_filename = f"cover_letter_job_{job_id}.pdf"
        res_pdf = save_text_as_pdf_method(updated_cover, filename=pdf_filename)
        if res_pdf.get("status") != "success":
            print("ERROR saving PDF:", res_pdf.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "save_pdf"})
            continue
        pdf_path = res_pdf["file_path"]
        print(f"✅ Saved updated cover letter: {pdf_path}")

        # Print email
        email_body = f"I am writing to express interest in the {job_title} role. Please find my updated cover letter attached for your review."
        res_email = prepare_and_print_email_method(
            recruiter_email=recruiter_email,
            recruiter_name=recruiter_name,
            job_title=job_title,
            email_body=email_body,
            pdf_path=pdf_path
        )
        out_all.append({"job_index": i, "status": res_email.get("status"), "pdf_path": pdf_path})

    print("\nPipeline complete.")
    return {"status": "finished", "results": out_all}



In [52]:
# ------------------ Run Pipeline -------------------
print("⏳ Running CoMail AI Orchestrator Pipeline (no email sent)...")
pipeline_result = orchestrator_pipeline()
print("✅ Pipeline finished. Summary:")
print(pipeline_result)


⏳ Running CoMail AI Orchestrator Pipeline (no email sent)...
Found 3 job rows. Processing sequentially...

--- Processing job 1/3:  (id=1) ---
✅ Saved updated cover letter: /kaggle/working/cover_letter_job_1.pdf

📧  EMAIL PREVIEW
To: rachael@reatil.com
Subject: Application for 

--- Email Body ---

Dear rachael x,

I am writing to express interest in the  role. Please find my updated cover letter attached for your review.

Best regards,
Sanya

--- Attachment (cover letter PDF saved) ---
/kaggle/working/cover_letter_job_1.pdf


--- Processing job 2/3:  (id=2) ---
✅ Saved updated cover letter: /kaggle/working/cover_letter_job_2.pdf

📧  EMAIL PREVIEW
To: ross@museum.com
Subject: Application for 

--- Email Body ---

Dear ross y,

I am writing to express interest in the  role. Please find my updated cover letter attached for your review.

Best regards,
Sanya

--- Attachment (cover letter PDF saved) ---
/kaggle/working/cover_letter_job_2.pdf


--- Processing job 3/3:  (id=3) ---
✅ Saved upd

In [53]:
!ls -lah /kaggle/working/

total 24K
drwxr-xr-x 3 root root 4.0K Dec  1 05:20 .
drwxr-xr-x 5 root root 4.0K Dec  1 03:40 ..
-rw-r--r-- 1 root root 2.4K Dec  1 05:20 cover_letter_job_1.pdf
-rw-r--r-- 1 root root 2.4K Dec  1 05:20 cover_letter_job_2.pdf
-rw-r--r-- 1 root root 2.4K Dec  1 05:20 cover_letter_job_3.pdf
drwxr-xr-x 2 root root 4.0K Dec  1 03:41 .virtual_documents
